In [1]:
import os
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = "1" #use GPU:0 only
gpus = tf.config.experimental.list_physical_devices(device_type='GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(device=gpu,enable=True) 

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
#print GPU Devices
[print(x) for x in local_device_protos if x.device_type == 'GPU']
print('Tensorflow Version:',tf.__version__)

In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/qzhang/anaconda3/lib/python3.7/site-packages/matplo

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10561793600
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 2443545207557066409
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5"

Tensorflow Version: 2.4.1


# Preproecessing

In [2]:
# load_data
data_2duv = np.load('/data1/qzhang/small_dataset_pnas/original/original_dataset.npz')['twoduv']
labels = np.load('/data1/qzhang/small_dataset_pnas/original/original_dataset.npz')['labels']

# oever view
print('Original 2duv shape:',data_2duv.shape)
print('labels:',np.unique(labels))

Original 2duv shape: (2700, 25921)
labels: ['a-helix' 'b-sheet' 'other-SS']


In [3]:
# preprocess labels
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels_cat = encoder.fit_transform(labels)
print(encoder.classes_)
print(np.unique(labels_cat))

['a-helix' 'b-sheet' 'other-SS']
[0 1 2]


In [4]:
# preprocess 2duv data

# split dataset
from sklearn.model_selection import train_test_split
train_data,test_data,train_label,test_label = train_test_split(data_2duv.reshape(-1,161,161,1),labels_cat,
                                                               stratify = labels_cat,
                                                               test_size=0.2,
                                                               random_state=42)

train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,stratify=train_label,
                                               test_size = 0.125,random_state=42)


print('Train data:',train_x.shape)
print('Validation data:',val_x.shape)
print('Test data:',test_data.shape)


Train data: (1890, 161, 161, 1)
Validation data: (270, 161, 161, 1)
Test data: (540, 161, 161, 1)


In [6]:
# prprocess 2duv data

mean = train_x.mean()
std = train_x.std()

train_x -= mean
train_x /= std

print('Mean value:',mean)
print('Std value:',std)

print('Train datset after nomolization,Mean:%f,Std:%f'%(train_x.mean(),train_x.std()))

train_x = train_x.clip(-2,2)
print('Max:%d, Min:%d'%(train_x.max(),train_x.min()))


# preprocess val_data and test_data
val_x -= mean
val_x /= std

val_x = val_x.clip(-2,2)

test_data -= mean
test_data /= std

test_data = test_data.clip(-2,2)

Mean value: -1.59583661486646e-24
Std value: 1.1228893279331382e-21
Train datset after nomolization,Mean:0.000000,Std:1.000000
Max:2, Min:-2


# Build  a 2D CNN Demo

In [8]:
#建立模型
from tensorflow.keras import models,layers,optimizers
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=(161,161,1)))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(32, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['acc'])
    return model


In [9]:
model = build_model()
history = model.fit(train_x,
                   train_y, epochs=50, batch_size=64,
                   validation_data=(val_x,val_y))

Epoch 1/50
30/30 [==============================] - 5s 41ms/step - loss: 1.0321 - acc: 0.4709 - val_loss: 1.0353 - val_acc: 0.6481
Epoch 2/50
30/30 [==============================] - 0s 15ms/step - loss: 0.8382 - acc: 0.7411 - val_loss: 1.2814 - val_acc: 0.7037
Epoch 3/50
30/30 [==============================] - 0s 16ms/step - loss: 0.5146 - acc: 0.8400 - val_loss: 2.3227 - val_acc: 0.7037
Epoch 4/50
30/30 [==============================] - 0s 16ms/step - loss: 0.3417 - acc: 0.8936 - val_loss: 3.5639 - val_acc: 0.7556
Epoch 5/50
30/30 [==============================] - 0s 16ms/step - loss: 0.2660 - acc: 0.9067 - val_loss: 3.7984 - val_acc: 0.7259
Epoch 6/50
30/30 [==============================] - 0s 15ms/step - loss: 0.2443 - acc: 0.9130 - val_loss: 3.7186 - val_acc: 0.7481
Epoch 7/50
30/30 [==============================] - 0s 15ms/step - loss: 0.1745 - acc: 0.9510 - val_loss: 3.8123 - val_acc: 0.7407
Epoch 8/50
30/30 [==============================] - 0s 15ms/step - loss: 0.1690 - a

### Test test_data

In [10]:
model.evaluate(test_data,test_label)

17/17 [==============================] - 0s 9ms/step - loss: 5.5715 - acc: 0.8370


[5.571462631225586, 0.8370370268821716]

### save model

In [11]:
model.save('./2DCNNDemoModel.h5')